<a href="https://colab.research.google.com/github/Dhaneshkp/NLP/blob/main/Langchain_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
from langchain_astradb import AstraDBVectorStore
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv
import os
import pandas as pd
#from google.ai import gemini
import pandas as pd
from langchain_core.documents import Document
from google.colab import userdata
from langchain.embeddings.openai import OpenAIEmbeddings

def dataconveter():
    product_data=pd.read_csv("flipkart_product_review.csv")

    data=product_data[["product_title","review"]]

    product_list = []

    # Iterate over the rows of the DataFrame
    for index, row in data.iterrows():
        # Construct an object with 'product_name' and 'review' attributes
        obj = {
                'product_name': row['product_title'],
                'review': row['review']
            }
        # Append the object to the list
        product_list.append(obj)



    docs = []
    for entry in product_list:
        metadata = {"product_name": entry['product_name']}
        doc = Document(page_content=entry['review'], metadata=metadata)
        docs.append(doc)
    return docs



In [31]:

load_dotenv()

GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")
ASTRA_DB_API_ENDPOINT=userdata.get("ASTRA_DB_API_ENDPOINT")
ASTRA_DB_APPLICATION_TOKEN=userdata.get("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_KEYSPACE=userdata.get("ASTRA_DB_KEYSPACE")
YOUR_OPENAI_API_KEY=userdata.get("OPENAI_API_KEY")

In [42]:
#!pip install google-cloud-aiplatform langchain

In [69]:
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModel

    # Choose your model

model_name = "sentence-transformers/all-MiniLM-L6-v2"

    # Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

    # Create HuggingFaceEmbeddings object
embedding = HuggingFaceEmbeddings(  model_name=model_name)

In [34]:
#embedding = OpenAIEmbeddings(openai_api_key='YOUR_OPENAI_API_KEY')
#embedding=GoogleGenerativeAIEmbeddings(google_api_key=GOOGLE_API_KEY,model='models/embedding-gecko-001')

In [75]:
from langchain.vectorstores import FAISS
#from google.ai import gemini


def ingestdata(status):


    storage=status

    if storage==None:
        docs=dataconveter()
        db = FAISS.from_documents(docs, embedding)
    else:
        return db
    return db

db=ingestdata(None)

docs = db.similarity_search("can you tell me the low budget sound basshead.")
for doc in docs:
    print(f"{doc.page_content} \n\nMetadata: {doc.metadata}")





If you are bass lover don't dare to buy this. Bass is really very bad. I have used Sony earphones and they offer pretty good bass than it. I bought this due to hype but in actuality it's not even close to my expectations. Whenever i listen to music it makes me  🤬 

Metadata: {'product_name': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset'}
Best sound & bass quality 

Metadata: {'product_name': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset'}
The bass provided is a decent one and the sound clarity is pretty good.And all the other features are exceptional.U can surely go for this one for the price range..will not regeret. 

Metadata: {'product_name': 'OnePlus Bullets Wireless Z Bluetooth Headset'}
Best in this price Bass 4.5*Sound 4.5*Battery backup 4.9*Design 4.2* 

Metadata: {'product_name': 'U&I Titanic Series - Low Price Bluetooth Neckband Bluetooth Headset'}


In [70]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load tokenizer and model
model_name = "sentence-transformers/all-MiniLM-L6-v2"#"facebook/bart-base"  # Replace with your desired Bart model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create HuggingFaceEmbeddings object
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Assuming 'db' is your FAISS database created with the above embedding
retriever = db.as_retriever(search_kwargs={"k": 3})

PRODUCT_BOT_TEMPLATE = """
Your ecommercebot bot is an expert in product recommendations and customer queries.
It analyzes product titles and reviews to provide accurate and helpful responses.
Ensure your answers are relevant to the product context and refrain from straying off-topic.
Your responses should be concise and informative.

CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:

"""

prompt = ChatPromptTemplate.from_template(PRODUCT_BOT_TEMPLATE)

# Create a text generation pipeline using BartForCausalLM
generator = pipeline(
    "text2text-generation",  # Use text2text-generation for Bart
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50  # Adjust as needed
)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    # Extract the 'content' string from the HumanMessage object
    | (lambda x: x.to_messages()[0].content)
    | generator
    |(lambda x: x[0]['generated_text'])
    | StrOutputParser()
)

print(chain.invoke("can you tell me the best bluetooth buds?"))

If you want to use `BertLMHeadModel` as a standalone, add `is_decoder=True.`
Some weights of BertLMHeadModel were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L6-v2 and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
The model 'BertLMHeadModel' is not supported for text2text-generation. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGene

your ecommercebot bot is an expert in product recommendations and customer queries. it analyzes product titles and reviews to provide accurate and helpful responses. ensure your answers are relevant to the product context and refrain from straying off - topic. your responses should be concise and informative. context : [ document ( metadata = { ' product _ name ' : ' oneplus bullets wireless z bass edition bluetooth headset ' }, page _ content = ' best earphone ' ), document ( metadata = { ' product _ name ' : ' oneplus bullets wireless z bluetooth headset ' }, page _ content = ' best budget bluetooth earphones by one plus.... sound quality superb..... bass also good.... vry good built quality.... fix 15 hours play back.... very fast order delivery by flipkart.... over all good ' ), document ( metadata = { ' product _ name ' : ' realme buds q bluetooth headset ' }, page _ content = ' best quality buds ' ) ] question : can you tell me the best bluetooth buds? your answer : york corner c

In [74]:
# Install the necessary packages for Llama 2
!pip install -q transformers accelerate bitsandbytes

# Import the necessary modules from transformers
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

# Load the quantized Llama 2 model
model_id = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id)

# Assuming 'db' is your FAISS database created with the above embedding
retriever = db.as_retriever(search_kwargs={"k": 3})

PRODUCT_BOT_TEMPLATE = """
Your ecommercebot bot is an expert in product recommendations and customer queries.
It analyzes product titles and reviews to provide accurate and helpful responses.
Ensure your answers are relevant to the product context and refrain from straying off-topic.
Your responses should be concise and informative.

CONTEXT:
{context}

QUESTION: {question}

YOUR ANSWER:

"""

prompt = ChatPromptTemplate.from_template(PRODUCT_BOT_TEMPLATE)

from langchain import HuggingFacePipeline
from transformers import pipeline

# Create a text generation pipeline using Llama 2
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=50,
    temperature=0.7
)

llm = HuggingFacePipeline(pipeline=pipe)

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    # Extract the 'content' string from the HumanMessage object
    | (lambda x: x.to_messages()[0].content)
    | llm
    | StrOutputParser()
)

print(chain.invoke("can you tell me the best bluetooth buds?"))


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



Your ecommercebot bot is an expert in product recommendations and customer queries.
It analyzes product titles and reviews to provide accurate and helpful responses.
Ensure your answers are relevant to the product context and refrain from straying off-topic.
Your responses should be concise and informative.

CONTEXT:
[Document(metadata={'product_name': 'OnePlus Bullets Wireless Z Bass Edition Bluetooth Headset'}, page_content='Best earphone'), Document(metadata={'product_name': 'OnePlus Bullets Wireless Z Bluetooth Headset'}, page_content='Best budget Bluetooth earphones by one plus ....sound quality superb.....bass also good .... vry good built quality ....fix 15 hours play back ....very fast order delivery by flipkart ....over all good'), Document(metadata={'product_name': 'realme Buds Q Bluetooth Headset'}, page_content='Best Quality buds')]

QUESTION: can you tell me the best bluetooth buds?

YOUR ANSWER:


Your ecommerce bot will use a combination of a "listening system" and a "b

In [49]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [53]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 38.7 MB/s eta 0:00:00


In [5]:
!pip install  langchain-astradb langchain langchain-openai datasets pypdf python-dotenv flask langchain_google_genai

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 19.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 5.5 MB/s eta 0:00:0

In [1]:
!pip install openai tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00


In [46]:
!pip install langchain_community
!pip install transformers
!pip install langchain